### 괴담 종류 이진분류: E 타입(Experiment), G 타입(Guess) <hr>

1. 데이터 전처리, 인코딩
2. 모델 구현


In [32]:
# 모듈 로딩
import torch
import pandas as pd
#!pip install openpyxl

In [28]:
data1  =pd.read_excel('2조 괴담 파일.xlsx', sheet_name='BJY', header=1)
data2 = pd.read_excel('2조 괴담 파일.xlsx', sheet_name='LHG_Neapolitan', header= 1)
data3 = pd.read_excel('2조 괴담 파일.xlsx', sheet_name='LHG_Horror', header= 1)
data4 = pd.read_excel('2조 괴담 파일.xlsx', sheet_name='SY', header= 1)

In [29]:
horror = pd.concat([data1, data2, data3, data4], axis=0)

In [30]:
horror.columns

Index(['CLASS', 'TYPE', 'FROM', 'TEXT'], dtype='object')

In [31]:
horror = horror.drop(['CLASS', 'FROM'], axis = 1)

In [33]:
horror.head()

,TYPE,TEXT
0,E,"""그런데 어떻게 알았어?"" 이렇게 묻더라구요, 저는 뭐가? 라고 대답하니 ""꿈인줄 ..."
1,E,또 똑같은 꿈을 꿨습니다. 방에서 일어나니까 옆방에서 여자가 미친듯이 웃으면서 00...
2,E,그리고 출근 한 다음 근무 후 새벽에 퇴근하고 아침에 잠들었는데 낮 열두시쯤 까지 ...
3,E,여튼 때는 20대 중반이었나?? 여름이었던것 같은데 여느때처럼 내방에서 잠을 자고 ...
4,E,온몸에 소름이 돋으면서 식은땀 ㅈㄴ나고 뭐지?? 도둑인가?? 이런 생각에 너무 무서...


In [41]:

horror.isnull().sum()
horror.isna().sum()

TYPE    0
TEXT    0
dtype: int64

In [43]:
# 균형데이터인지 체크하기
horror['TYPE'].value_counts()

TYPE
G    1806
E    1394
Name: count, dtype: int64

In [44]:
# feature / label 분리

labelSR = horror['TYPE']
horrorDF = horror['TEXT'].to_frame()

print(f'[HORROR] {horrorDF.shape}, [LABEL] {labelSR}')

[HORROR] (3200, 1), [LABEL] 0      E
1      E
2      E
3      E
4      E
      ..
994    E
995    E
996    E
997    E
998    E
Name: TYPE, Length: 3200, dtype: object


## 정제하기
- 노이즈데이터, 불용어 제거하기
- 토큰화하기
- 정수화하기

In [46]:
import string

punctuation = string.punctuation
len(punctuation)
hangle_pattern = "[^ㄱ-ㅎㅏ-ㅣ가-힣]"
number_pattern = "[0-9]"
eng_pattern = "[a-zA-Z]"


In [47]:
# 한글, 영어 숫자 제외한 다른 문자 제거하기
horrorDF['TEXT'] = horrorDF['TEXT'].str.replace(hangle_pattern, '', regex=True)
horrorDF['TEXT'] = horrorDF['TEXT'].str.replace(number_pattern, '', regex = True)
horrorDF['TEXT'] = horrorDF['TEXT'].str.replace(eng_pattern, '', regex= True)

In [48]:
horrorDF['TEXT'][:5]

0    그런데어떻게알았어이렇게묻더라구요저는뭐가라고대답하니꿈인줄어떻게알았어이렇게물어서느낌이싸...
1    또똑같은꿈을꿨습니다방에서일어나니까옆방에서여자가미친듯이웃으면서아다시온다고했지ㅋㅋㅋ이러...
2    그리고출근한다음근무후새벽에퇴근하고아침에잠들었는데낮열두시쯤까지꿈은안꿨습니다잠깐잠에서깨...
3    여튼때는대중반이었나여름이었던것같은데여느때처럼내방에서잠을자고있었음자다가새벽에깻는데눈은...
4    온몸에소름이돋으면서식은땀ㅈㄴ나고뭐지도둑인가이런생각에너무무서워서눈뜨면날죽일것같다는생각...
Name: TEXT, dtype: object

In [70]:
from sklearn.model_selection import train_test_split




In [50]:
import numpy as np
#horrorDF['TEXT'] = horrorDF['TEXT'].str.replace('^+',"", regex=True)
horrorDF['TEXT'].replace('', np.nan, inplace=True)

horrorDF['TEXT'] = horrorDF['TEXT'].dropna()
horrorDF.isna().sum()

TEXT    0
dtype: int64

In [52]:
from urllib.request import urlretrieve

filename = '../datas/kr_stopword.txt'
url = "https://gist.githubusercontent.com/chulgil/d10b18575a73778da4bc83853385465c/raw/a1a451421097fa9a93179cb1f1f0dc392f1f9da9/stopwords.txt"
ret = urlretrieve(url, filename)

In [99]:
## 토큰화

from konlpy.tag import Okt
okt = Okt()
result = horror.TEXT.str.cat(horror['TEXT'])
result

Xtrain, Xtest, ytrain, ytest = train_test_split(horrorDF['TEXT'], labelSR, test_size = 0.2, random_state=42)
train = pd.concat([Xtrain, ytrain], axis=1)
test_= pd.concat([Xtest, ytest], axis=1)

train_tokens = [okt.morphs(sentence) for sentence in train['TEXT']]
test_tokens = [okt.morphs(sentence) for sentence in test_['TEXT']]

In [121]:
train_tokens = [[train_tokens]]
test_tokens = [[test_tokens]]

In [122]:
vocab = {}
for idx in range(horrorDF.shape[0]):
    result = okt.morphs(horrorDF.iloc[idx][0])
    for word in result:
        if vocab.get(word):
            vocab[word] += 1
        else:
            vocab[word] = 1

# vocab = pd.Series(vocab).to_frame()

In [124]:
# sorted(vocab, key= lambda a:a[1])

In [125]:
from soynlp.normalizer import *


In [126]:
# 불용어 처리

stopwords = set(open(filename). read().splitlines())

clean = {token: value for token, value in vocab.items() if token not in stopwords}
clean

{'이렇게': 86,
 '묻더라구요': 1,
 '는': 2440,
 '뭐': 133,
 '가라': 4,
 '고': 454,
 '대답': 97,
 '니': 176,
 '꿈인줄': 1,
 '물어': 16,
 '서': 306,
 '느낌': 78,
 '싸해': 1,
 '돌아보니': 6,
 '여자친구': 19,
 '얼굴': 177,
 '아니라': 33,
 '노이즈': 1,
 '낀것처럼': 1,
 '기억': 103,
 '안나': 22,
 '만': 242,
 '받았습니다': 3,
 '뭔가': 81,
 '잘못': 41,
 '됬다싶어서': 1,
 '입술': 14,
 '다시': 135,
 '깨물고': 2,
 '잠': 78,
 '확': 6,
 '깼는데': 3,
 '출근시간': 2,
 '도': 889,
 '아직': 65,
 '멀었길래': 1,
 '그냥': 118,
 '개': 131,
 '꿈': 63,
 '이다': 155,
 '생각': 475,
 '하고': 566,
 '더': 141,
 '잤습니다': 1,
 '아니나': 2,
 '다를까': 2,
 '똑같': 2,
 '이방': 13,
 '일어나는거로': 1,
 '시작': 215,
 '했는데': 79,
 '옆방': 7,
 '여자': 172,
 '미친듯이': 13,
 '웃으면서': 12,
 '이름': 68,
 '왔어': 6,
 'ㅋㅋㅋ': 11,
 '식': 53,
 '웃고있더라구요': 1,
 '깨려고': 2,
 '막': 26,
 '깨물고있는데': 1,
 '어차피': 11,
 '올꺼야': 1,
 'ㅋㅋ': 13,
 '이따': 2,
 '얘기': 105,
 '했습니다': 139,
 '깬': 6,
 '남았고': 1,
 '누워서': 3,
 '핸드폰': 14,
 '다가': 23,
 '모르게': 8,
 '잠들었는데': 5,
 '똑같은': 12,
 '꿨습니다': 2,
 '방': 132,
 '일어나니까': 2,
 '다': 632,
 '시온': 1,
 '했지': 23,
 '이러면서': 2,
 '제방': 4,
 '뛰어오더라구요': 1,
 '깨물어도': 1,


In [127]:
vocab = pd.Series(clean).to_frame()

vocab.head()

,0
이렇게,86
묻더라구요,1
는,2440
뭐,133
가라,4


In [128]:
vocab = pd.Series(clean).to_frame()

vocab.head()

,0
이렇게,86
묻더라구요,1
는,2440
뭐,133
가라,4


In [129]:
# 인코딩 : 문자 -> 숫자
encode = {token: idx for idx, token in enumerate(vocab)}

# 디코딩 : 숫자 -> 문자
decode = {idx: token for idx, token in enumerate(vocab)}


In [130]:
#인코딩
unk_id = encode.get(UNK)

train_id = [[encode.get(token, unk_id) for token in text]for text in train_tokens] #[[encode.get(token, unk_id) for token in text]for text in ytrain]
test_id = [[encode.get(token, unk_id) for token in text]for text in test_tokens] #[[encode.get(token, unk_id) for token in text]for text in ytest]

TypeError: unhashable type: 'list'

[[None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None]]

In [115]:
def pad_sequences(sentences, max_length, pad, start='R'):
    result = []
    for sen in sentences:
        sen = sen[:max_length] if start == 'L' else sen[-1 * max_length:]
        padd_sen = sen + [pad] * (max_length - len(sen)) if start =='R' else [pad] * (max_length - len(sen)) + sen
        result.append(padd_sen)
    return result


In [116]:
PAD_ID = encode.get(PAD)
MAX_LENGTH = 32
train_id = pad_sequences(train_id, MAX_LENGTH, PAD_ID)
test_id = pad_sequences(test_id, MAX_LENGTH, PAD_ID)

print(f'[TRAIN] {train_id[0]}, [TEST] {test_id[0]}')

[TRAIN] [None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None], [TEST] [None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]


In [113]:
#학습, 테스트용 데이터 패딩 처리하기
# PAD_ID = encode.get(PAD)
# MAX_LENGTH = 32

# train_id = pad_sequences(train_id, MAX_LENGTH, PAD_ID)
# test_id = pad_sequences(test_id, MAX_LENGTH, PAD_ID)

[]
[]
[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
[None, None]
[None, None]
[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]


In [104]:
print(f'[TRAIN] {train_id[0]}, [TEST] {test_id[0]}')

[TRAIN] [], [TEST] [None, None]
